In this excercise we are working with the Google Earth engine

In [33]:
import rioxarray as rio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import geemap
import ee
import os

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWjwi0Xp81xuba5G3ldFCZngdD-_Ka4XyroaTwzJYn2Zry-hjE2KOQE

Successfully saved authorization token.


In [3]:
# Load the ERA5 Monthly Aggregates 
# A description can be found here: https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY?hl=en#description
# We are only interested in the precipitation, so we select that band.

ERA5_precip = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('total_precipitation')

In [4]:
# We have a timeseries between 1979 and 2020-06, let's focus only on the summer precipitation of the last 10 years.
# An ImageCollection can be filtered between a certain date range using filterDate and by specifying a calander range
# a selection of months can be made. Try different months or date range, but try not to select too much data as it will take long to download

filterMonth = ee.Filter.calendarRange(7, 9, 'month');
ERA5_precip_range = ERA5_precip.filterDate('2010-01-01', '2020-01-01').filter(filterMonth)

In [5]:
"""
We are only interested in one precipitation map per year, so we average the ImageCollection over each summer.
To do this we need to take three steps:
 - Make a 'list' of unique years
 - group the ImageCollection per year using the unique year list
 - take the average over each group and save the resulting image
"""
# the unqiue year list is made by the 'distinct' feature
distinctYear = ERA5_precip_range.distinct('year');
# To create groups per year we need a filter that compares the year of each image to the unique years
filter = ee.Filter.equals(leftField= 'year',rightField= 'year');
# The groups are saved as 'sameYear'
join = ee.Join.saveAll('sameYear')

# Apply the join and label each image based on the year
joinCol = ee.ImageCollection(join.apply(distinctYear, ERA5_precip_range,filter))

In [ ]:
print(joinCol.getInfo())

In [6]:
def func_sma(img):
  yearCol = ee.ImageCollection.fromImages(
    img.get('sameYear')
  )
  return yearCol.mean().set('Year', img.get('year'))

annualMonthlyMeanCol = joinCol.map(func_sma)

In [ ]:
print(annualMonthlyMeanCol.getInfo())

In [7]:
# We only want to work with a small part of the data, so we load a shapefile containing the borders of all countries in the world
# Get a feature collection of administrative boundaries.
Bounds = ee.FeatureCollection('FAO/GAUL/2015/level2')

In [8]:
# To select a specific province we need to know the name. We can visualize the boundaries using the geemap Map Inspector
# set our initial map parameters for The Netherlands
center_lat = 52.36
center_lon = 4.89
zoomlevel=7

Map = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
Map.addLayer(Bounds)

Map
# By selecting the Inspector from the tools menu (right top in the map) we can click on a province to get information on the data
# We need the name of the feature that contains the names of the provinces, and the names of the provinces we want to select.

Map(center=[52.36, 4.89], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [9]:
# Now we know the name of the Provinces we want to select the precipitation data in the regions-of-interest
# Here we selected Friesland and Limburg, but try different regions (does not need to be in the Netherlands)

#Filtering is done with the .filter feature and a filter equation. From the inspector we can see that the feature 
# in the Bounds layer related to the provinces is called ADM1_NAME, so we use this in the equation.
Friesland = Bounds.filter(ee.Filter.eq('ADM1_NAME', 'Friesland'))
Limburg = Bounds.filter(ee.Filter.eq('ADM1_NAME', 'Limburg'))

In [10]:
# With the selected regions we can now cut out our data. As explained earlier to apply an operation such as clip to an ImageCollection
# we need to map it to all imagesin the collection using a function.

# In order to use the same functions for different regions we add the geometry as a variable and need to create a nested function. 
# The lambda notation is a concise way to create a function in the Python API 
def Collection_clip(collection,geom): 
    
    return collection.map(lambda image: image.clip(geom))

# An alternative form would be:
def Collection_clip2(collection,geom):
    def im_clip(imgs):
        return imgs.clip(geom)
    return collection.map(im_clip)

annualMonthlyMean_Frl = Collection_clip(annualMonthlyMeanCol,Friesland.geometry())
annualMonthlyMean_Lmb = Collection_clip(annualMonthlyMeanCol,Limburg.geometry())

In [11]:
# The precipitation data is still an ImageCollection that contains annually averaged precipitation data. 
# To visualize precipitation data on the map we have to reduce the ImageCollection to a single image using the average, for example 
# or for simplicity we can just select the first year.   

# In order to see the variation on the map we specify a min and max value for visualization. Use the inspector tool to see if these are
# chosen well.
vis = {
    'min': 0,
    'max': 0.15
}

# Here we select the first year (2010) and add it to the excisting map 
Map.addLayer(annualMonthlyMean_Frl.first(), vis)
Map.addLayer(annualMonthlyMean_Lmb.first(), vis)

In [ ]:
Limburg.getInfo()

In [ ]:
# In order to continue working with the data we download the data in two parts (one for each province)
# Each ImageCollection contains 10 images, we could export each image separately or merge the collection in one image with 10 bands

# To export an image we need to provide at least three things:
# - Region
# - scale (resolution)
# - filename
# - crs

# We can export data at different ways. The Earth engine way is to export is to your Google drive, from which you can download it to your PC.
# Export to drive
# To save the ImageCollection as one image we use .toBands()
"""
task = ee.batch.Export.image.toDrive(image=annualMonthlyMean_Lmb.toBands(),
                                     description='Precip_Lmb',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     folder="ASA",
                                     region=Limburg.geometry(),
                                     scale=27784,
                                     fileNamePrefix='Precip_Lmb',
                                     maxPixels= 3784216672400,)
task.start()
"""

# The second way is to use the geemap export. By first defining an ouput directory and filename the data is directly dowloaded to your PC.
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'Precip_Lmb.tif')
geemap.ee_export_image(annualMonthlyMean_Lmb.toBands(), filename=filename, scale=27784, region=Limburg.geometry(), file_per_band=False)